# ワークフロー実行のための準備

このノートブックでは、ワークフロー実行のための準備を行います。 
上から順番にセルを実行してください。

## 1. SSH鍵を準備する

1. SSH鍵を生成し、秘密鍵を.ssh/id_rsaに配置します。

In [ ]:
%%sh
cd ~/
mkdir .ssh
ssh-keygen -f .ssh/id_rsa -t rsa -N ""

2. 以下のセルで出力された公開鍵を[こちら](http://dg02.dg.rcos.nii.ac.jp/user/settings/ssh)からGINリポジトリに登録してください。

In [ ]:
%%sh
cd ~/
cat .ssh/id_rsa.pub

## 2. SSH接続で書き戻しを行うためにssh鍵に権限を与える（ssh鍵は.sshディレクトリの中にid_rsaとして保存する）

In [ ]:
%%sh
cd ~/
chmod 600 .ssh/id_rsa

## 3. SSH鍵の利用確認をオフにする

In [ ]:
%%sh
echo "host *\n\tStrictHostKeyChecking no\n\tUserKnownHostsFile /dev/null" >> ~/.ssh/config

## 4. .gitconfigにユーザー名とメールアドレスを登録する

    1. 以下のセルでgitアカウントのユーザー名とメールアドレスを入力してください。

In [ ]:
name = input()

In [ ]:
email = input()

    2. 入力されたユーザー名とメールアドレスを.gitconfigに登録します。

In [ ]:
import os

os.system("cd /home/jovyan")
os.system("git config --global user.name " + name)
os.system("git config --global user.email " + email)

## 5. Dataladを使用するため、このリポジトリをデータセットとする

In [ ]:
%%sh
cd ~/
datalad create --force

## 6. ワークフロー実行結果をGINリポジトリに反映させるため、GINリポジトリをリモートリポジトリに設定する

In [ ]:
%%sh
mkdir ./tmp
git remote show origin > ./tmp/remote.txt

In [ ]:
import json
json_open = open('/home/jovyan/WORKFLOW/param_files/param_for_dataladsiblings.json', 'r')
json_load = json.load(json_open)
http = json_load['http']
ssh = json_load['ssh']

In [ ]:
with open('./tmp/remote.txt', 'r') as f:
    lines = [s.strip() for s in f.readlines()]
    ssh_push_url = lines[2].replace(http, ssh) 

In [ ]:
import os

os.system("rm -r ./tmp")

In [ ]:
from datalad import api

try:
    api.siblings(action='add', name='gin', url=ssh_push_url)
except:
    pass

## 7. Git管理対象外ファイルを.gitignoreで設定する

In [ ]:
%%sh
echo "/.cache/" >> ~/.gitignore
echo "/.conda/" >> ~/.gitignore
echo "/.config/" >> ~/.gitignore
echo "/.ipython/" >> ~/.gitignore
echo "/.local/" >> ~/.gitignore
echo ".ipynb_checkpoints/" >> ~/.gitignore

## 8. 実験用フォルダを作成し、その中にpipeline.jsonを用意する

In [ ]:
%%sh
mkdir ~/experiments
echo "[]" >> ~/experiments/pipeline.json

## 9. 必要なファイルをPACKAGEテンプレートに複製する

In [ ]:
%%sh
cd ~/
mkdir WORKFLOW/PACKAGE/.ssh
cp .ssh/id_rsa WORKFLOW/PACKAGE/.ssh/id_rsa
cp .ssh/config WORKFLOW/PACKAGE/.ssh/config
cp .gitconfig WORKFLOW/PACKAGE/.gitconfig
cp .gitconfig WORKFLOW/PACKAGE/.gitconfig
cp environment.yml WORKFLOW/PACKAGE/environment.yml

## 10. 変更内容を書き戻す

In [ ]:
import papermill as pm

pm.execute_notebook(
    '../util/datalad_save_push.ipynb',
    '-',
    parameters = dict(SAVE_MESSAGE = 'ワークフロー実行準備')
)